# KAGLE COMPETITION - TITANIC ML from Disaster

### Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


### EDA

In [28]:
train = pd.read_csv('data/train.csv', index_col='PassengerId')

print(train.dtypes)

colummns_wnan = train.columns[train.isna().any()].tolist()
# See if missing data per columns >5% total instances

def clean_data(df):
    df = drop_nan_values(df)
    clean_df = clean_dtypes(df)
    return clean_df

def clean_nan_values(df):
    # % of max nans to drop
    max_nans = 0.05
    # Dropping the rows of those columns that have <5% of nans
    drop_nans = []
    # Replacing those nan values where its columns have >5% of nans
    replacing_nans =[]

    colummns_wnan = df.columns[df.isna().any()].tolist()
    len_df = len(df)
    for column in colummns_wnan:
        if df[column].isna().sum() > len_df*max_nans:
            replacing_nans.append(column)
        else:
            drop_nans.append(column)

    for drop_col in drop_nans:
        num_nan = df[column].isna().sum()
        print(f'Dropped {num_nan} rows due to nan values in {drop_col} column, corresponding to a {(num_nan/len_df)*100}% of the dataset length')
        df.drop(substet=[drop_col], inplace=True)



def clean_dtypes(df):
    return df


Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object


In [30]:
train.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [29]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
